# Mini Git - Basic Operations Demo

This notebook demonstrates the basic operations of Mini Git.

## Data Structures Used
- **Hash Table**: For file tracking and object storage
- **Tree**: For directory hierarchy
- **Linked List**: For commit history

In [1]:
import shutil
import os

# Clean up any existing test repository
test_dir = './test_repo'
if os.path.exists(test_dir):
    shutil.rmtree(test_dir)
print("Cleaned up old test repository")

Cleaned up old test repository


In [2]:
import sys
import os

# Add parent directory to path
sys.path.insert(0, os.path.abspath('..'))

from src.repository import Repository

## 1. Initialize Repository

Creates the `.minigit` directory structure using directory tree.

In [3]:
# Create test directory
test_dir = './test_repo'
os.makedirs(test_dir, exist_ok=True)

# Initialize repository
repo = Repository(test_dir)
if repo.init(author='Test User'):
    print("✓ Repository initialized successfully")
    print(f"  Location: {repo.minigit_path}")
else:
    print("Repository already exists")

✓ Repository initialized successfully
  Location: c:\Users\Admin\OneDrive\Desktop\JAY\DSA USING PYTHON\MINI GIT\notebooks\test_repo\.minigit


## 2. Create and Add Files

**Hash Table**: Files are tracked using SHA-1 hashing for content-addressable storage.

In [4]:
# Create test files
file1 = os.path.join(test_dir, 'file1.txt')
file2 = os.path.join(test_dir, 'file2.txt')

with open(file1, 'w') as f:
    f.write('Hello, Mini Git!\n')
    f.write('This is file 1')

with open(file2, 'w') as f:
    f.write('This is file 2\n')
    f.write('Another line')

print("✓ Created test files")

✓ Created test files


In [5]:
# Add files to staging area (Hash Table)
results = repo.add(['file1.txt', 'file2.txt'])

for file_path, success in results.items():
    if success:
        print(f"✓ Added: {file_path}")
        # Show the hash
        hash_value = repo.staging_area.get_file_hash(file_path)
        print(f"  SHA-1: {hash_value[:12]}...")

✓ Added: file1.txt
  SHA-1: 33f4f810d59f...
✓ Added: file2.txt
  SHA-1: 67f8a656ae30...


## 3. Check Status

**Hash Table Lookup**: O(1) time complexity for checking file status.

In [6]:
status = repo.status()

print("Repository Status:")
print(f"Branch: {repo.branch_manager.get_current_branch()}\n")

if status['staged_new']:
    print("Staged (new):")
    for path in status['staged_new']:
        print(f"  ✓ {path}")

print(f"\nTotal staged files: {repo.staging_area.get_file_count()}")

Repository Status:
Branch: main

Staged (new):
  ✓ file1.txt
  ✓ file2.txt

Total staged files: 2


## 4. Create Commit

**Linked List**: Commit history forms a singly linked list with parent pointers.

**Tree**: Directory structure is stored as an N-ary tree.

In [7]:
# Create first commit
commit_hash = repo.commit("Initial commit: Add files")

if commit_hash:
    print(f"✓ Created commit: {commit_hash[:8]}")
    
    # Get commit details
    commit = repo.commit_history.get_commit(commit_hash)
    print(f"  Author: {commit.author}")
    print(f"  Message: {commit.message}")
    print(f"  Parent: {commit.parent_hash or 'None (root commit)'}")
else:
    print("Nothing to commit")

✓ Created commit: 70cbc672
  Author: Test User
  Message: Initial commit: Add files
  Parent: None (root commit)


## 5. View Commit History

**Linked List Traversal**: Following parent pointers from head to tail.

In [8]:
commits = repo.log()

print("Commit History:")
print("=" * 50)
for commit in commits:
    print(commit)

# Show linked list structure
print("\nLinked List Structure:")
current_hash = repo.branch_manager.get_current_commit()
count = 0
while current_hash and count < 5:
    commit = repo.commit_history.get_commit(current_hash)
    print(f"{commit_hash[:8]} -> {commit.parent_hash[:8] if commit.parent_hash else 'NULL'}")
    current_hash = commit.parent_hash
    count += 1

Commit History:
Commit: 70cbc672c85572196bf8959f67e88ad74893661f
Author: Test User
Date: 2025-11-28 23:50:56

    Initial commit: Add files


Linked List Structure:
70cbc672 -> NULL


## 6. Modify Files and Show Diff

**Dynamic Programming**: LCS algorithm for computing diffs.

In [9]:
# Modify file1
with open(file1, 'w') as f:
    f.write('Hello, Mini Git!\n')
    f.write('This is file 1\n')
    f.write('NEW LINE ADDED')  # New line

print("Modified file1.txt\n")

# Show diff
diff = repo.diff('file1.txt')
print("Diff (using LCS algorithm):")
print("=" * 50)
print(diff)

Modified file1.txt

Diff (using LCS algorithm):
  Hello, Mini Git!
  This is file 1
+ NEW LINE ADDED


In [10]:
# Add and commit the change
repo.add(['file1.txt'])
commit_hash2 = repo.commit("Modified file1")
print(f"✓ Created commit: {commit_hash2[:8]}")

# Show updated history (linked list now has 2 nodes)
commits = repo.log()
print(f"\nTotal commits: {len(commits)}")
print("Linked list length: 2")

✓ Created commit: e58d314b

Total commits: 2
Linked list length: 2


## 7. Verify Hash Table Efficiency

Demonstrate O(1) lookup time for file operations.

In [11]:
import time

# Add multiple files
for i in range(10):
    filepath = os.path.join(test_dir, f'test_{i}.txt')
    with open(filepath, 'w') as f:
        f.write(f'Test file {i}')

files = [f'test_{i}.txt' for i in range(10)]

# Time the add operation
start = time.time()
repo.add(files)
end = time.time()

print(f"Added 10 files in {(end-start)*1000:.2f}ms")
print(f"Hash table size: {repo.staging_area.get_file_count()} files")

# Test lookup speed
start = time.time()
for i in range(1000):
    hash_val = repo.staging_area.get_file_hash('test_5.txt')
end = time.time()

print(f"1000 hash table lookups in {(end-start)*1000:.2f}ms")
print("✓ Demonstrates O(1) lookup complexity")

Added 10 files in 8.71ms
Hash table size: 10 files
1000 hash table lookups in 0.19ms
✓ Demonstrates O(1) lookup complexity


## Summary

This demo showed:
1. **Hash Tables**: Fast O(1) file tracking and object storage
2. **Trees**: Hierarchical directory structure representation
3. **Linked Lists**: Commit history with parent pointers
4. **Dynamic Programming**: LCS algorithm for diffs

All operations work efficiently with proper time complexities!

In [12]:
# Cleanup
import shutil
# shutil.rmtree(test_dir)  # Uncomment to clean up
print("Demo complete! (Uncomment cleanup to remove test_repo)")

Demo complete! (Uncomment cleanup to remove test_repo)
